In [ ]:
import os
import numpy
import pandas

from capblood_seq import config

from scrapi.dataset import Gene_Expression_Dataset as GED

In [ ]:
cell_type_labels = []

for cell_type in config.CELL_TYPES:
    
    if cell_type in config.CELL_SUBTYPES:

        for cell_subtype in config.CELL_SUBTYPES[cell_type]:

            cell_type_labels.append("%s %s" % (cell_subtype, cell_type))
    else:
        cell_type_labels.append(cell_type)

df = pandas.DataFrame(columns=cell_type_labels)

for sample in config.SAMPLE_NAMES:
    
    _, label_barcodes = GED.get_label_cells_from_file(os.path.join("data", sample, "labels.csv"))
    
    for subject_id in config.SUBJECT_IDS:
        
        if subject_id not in label_barcodes:
            continue
        
        series = pandas.Series(index=cell_type_labels, dtype=numpy.float32)
        
        for cell_type in cell_type_labels:
            series[cell_type] = len(set(label_barcodes[cell_type]).intersection(label_barcodes[subject_id]))
        
        df.loc["%s %s" % (subject_id, sample)] = series
df

In [ ]:
df_percentiles = df.divide(df.sum(axis=1), axis=0)*100
df_percentiles.to_csv(os.path.join("data", "capillary_cell_type_percentages.csv"))
df_percentiles